#### Lecture 5 - Integration: Romberg, Adaptive quadrature, Gauss quadrature

1. Implement these three methods to estimate: 
$$ \int_0^{0.8} 0.2+25𝑥−200𝑥^2+675𝑥^3−900𝑥^4+400𝑥^5 dx $$

The exact value is 1.640533. Use our `trapezoid` function from last time.

Test against built-in `scipy.integrate` functions.


In [1]:
import numpy as np

def f(x):
    return 0.2 + 25*x - 200*x**2 + 675*x**3 - 900*x**4 + 400*x**5

def trapezoid(f,a,b,n):
    x = np.linspace(a,b,n+1) # n segments, n+1 points between a and b
    I = (b-a)/(2*n) * (f(x[0]) + 2*np.sum(f(x[1:-1])) + f(x[-1]))
    return I

In [2]:
def romberg(f, a, b, tol=1e-4, maxiter=10):
    # estimate integral of f(x) from a to b
    # stop when tolerance reached or maxiter exceeded
    
    R = np.zeros((maxiter, maxiter))  # Romberg matrix
    n = 1 # first step size: (b-a), one segment
    
    R[0,0] = trapezoid(f,a,b,n)
    
    # Add a new row every iteration, and update columns
    for iter in range(1, maxiter):
        n = n * 2 # every new row uses 2x the segments for the trapezoid rule
        R[iter, 0] = trapezoid(f,a,b,n)  

        # update all other columns
        for k in range(1,iter+1):
            j = iter - k # Indexing hack to place the new value
            R[j,k] = (4**k * R[j+1,k-1] - R[j,k-1]) / (4**k - 1)
        
        # how much has the top-right value changed from the last column?
        if abs(R[0,k] - R[0,k-1]) < tol:
            break
    
    I = R[0,k]
    num_evals = n+1
    return I, num_evals

In [3]:
a = 0
b = 0.8
I, num_evals = romberg(f, a, b, tol=1e-4)
print('Romberg: I=%0.7f, num_evals = %d' % (I, num_evals))

Romberg: I=1.6405333, num_evals = 9


Compare to built-in function. If we want to see the full matrix, run with `show=True`. The full documentation is [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.romberg.html#scipy.integrate.romberg).

In [4]:
from scipy import integrate
result = integrate.romberg(f, a, b, tol=1e-4)
print(result) # print answer only
integrate.romberg(f, a, b, tol=1e-4, show=True) # print full matrix

1.6405333333333363
Romberg integration of <function vectorize1.<locals>.vfunc at 0x7fa8dedca9d0> from [0, 0.8]

 Steps  StepSize   Results
     1  0.800000  0.172800 
     2  0.400000  1.068800  1.367467 
     4  0.200000  1.484800  1.623467  1.640533 
     8  0.100000  1.600800  1.639467  1.640533  1.640533 

The final result is 1.6405333333333363 after 9 function evaluations.


1.6405333333333363

##### Adaptive quadrature

In [5]:
def quadapt(f, a, b, tol=1e-4):
    # returns integral estimate and number of evaluations
    I1 = trapezoid(f, a, b, 1)
    I2 = trapezoid(f, a, b, 2)
    
    if abs(I2-I1) < tol:
        return (4*I2 - I1)/3, 2+1 # romberg correction
    else:
        midpoint = (a+b)/2
        Ia,na = quadapt(f, a, midpoint, tol)
        Ib,nb = quadapt(f, midpoint, b, tol)
        return Ia + Ib, na+nb

In [6]:
I,num_evals = quadapt(f, a, b, tol=1e-4)
print('Adaptive: I=%0.7f, num_evals = %d' % (I, num_evals))

Adaptive: I=1.6405331, num_evals = 129


Compare to built in `quad`. Documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad)

In [7]:
result, err, infodict = integrate.quad(f, a, b, epsabs=1e-4, full_output=True)

In [8]:
print(result)

1.6405333333333307


In [9]:
print(err)

1.8213578793317207e-14


In [10]:
print(infodict['neval'])

21


The built-in function is much more efficient and accurate than our simplified version.

##### Gauss Quadrature

In [11]:
def gauss2(f,a,b):
    c = np.array([1,1])
    x = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])
    
    y = (a+b)/2 + (b-a)/2 * x  # map from [-1,1] to [a,b]
    I = (b-a)/2 * sum(c * f(y))  # estimate integral
    return I

In [12]:
I = gauss2(f,a,b)
print(I)

1.8225777777777772


We can write similar functions for 3, 4, 5, 6-point Gauss quadrature using the values from Table 22.1 of the textbook. For now, use the built-in `fixed_quad`. Specify `n=` the number of points. Docs [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.fixed_quad.html#scipy.integrate.fixed_quad).

In [13]:
for n in range(2,10):
    I,_ = integrate.fixed_quad(f, a, b, n=n)
    print('%d-point Gauss quadrature: %0.10f' % (n,I))

2-point Gauss quadrature: 1.8225777778
3-point Gauss quadrature: 1.6405333333
4-point Gauss quadrature: 1.6405333333
5-point Gauss quadrature: 1.6405333333
6-point Gauss quadrature: 1.6405333333
7-point Gauss quadrature: 1.6405333333
8-point Gauss quadrature: 1.6405333333
9-point Gauss quadrature: 1.6405333333


Our 2-point function matches this answer. Even though the 2-point estimate is not very accurate, the estimates with 3 and higher points are very accurate. 

Notice that 3-point Gauss should exactly integrate a $2N+1=5$th order polynomial, so in this case the answer is perfect. For a general $f(x)$ that is not a polynomial, it wouldn't work quite so well.

##### Compare accuracy to reach `tol=1e-4`
- Romberg: 9 function evaluations
- Adaptive: 21 function evaluations (with built-in function, ours was 129)
- Gauss: 3 function evaluations (special case for 5th order polynomial)

How do these compare to the trapezoid rule?

In [14]:
I_old = trapezoid(f, a, b, 1) # first estimate with one segment

ns = 2 ** np.arange(1, 12) # 2, 4, 8, etc segments

for n in ns:
    I = trapezoid(f, a, b, n)
    
    if abs(I - I_old) < 1e-4:
        break
    else:
        I_old = I


print('Trapezoid: I=%0.7f, num_evals = %d' % (I, n+1))

Trapezoid: I=1.6405236, num_evals = 513


We would need over 500 function evaluations with the trapezoid rule to reach a tolerance of `1e-4`. This can be obtained with fewer than 10 function evaluations using some of these more advanced integration methods.